# Stacking LSTM Layers
-----------------
Here we implement an LSTM model on all a data set of Shakespeare works. We will stack multiple LSTM models for a more accurate representation of Shakespearean language.  We will also use characters instead of words.

In [1]:
import os
import re
import string
import requests
import numpy as np
import collections
import random
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()

Start a computational graph session.

In [3]:
sess = tf.Session()

Set RNN Parameters

In [2]:
num_layers = 3 # Number of RNN layers stacked
min_word_freq = 5 # Trim the less frequent words off
rnn_size = 128 # RNN Model size, has to equal embedding size
epochs = 10 # Number of epochs to cycle through data
batch_size = 100 # Train on this many examples at once
learning_rate = 0.0005 # Learning rate
training_seq_len = 50 # how long of a word group to consider 
save_every = 500 # How often to save model checkpoints
eval_every = 50 # How often to evaluate the test sentences
prime_texts = ['thou art more', 'to be or not to', 'wherefore art thou']

Download/store Shakespeare data

In [4]:
data_dir = 'temp'
data_file = 'shakespeare.txt'
model_path = 'shakespeare_model'
full_model_dir = os.path.join(data_dir, model_path)

Declare the punctuation and then create the model and data directories

In [5]:
# Declare punctuation to remove, everything except hyphens and apostrophes
punctuation = string.punctuation
punctuation = ''.join([x for x in punctuation if x not in ['-', "'"]])

# Make Model Directory
if not os.path.exists(full_model_dir):
    os.makedirs(full_model_dir)

# Make data directory
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

Load the Shakespeare Data

In [6]:
print('Loading Shakespeare Data')
# Check if file is downloaded.
if not os.path.isfile(os.path.join(data_dir, data_file)):
    print('Not found, downloading Shakespeare texts from www.gutenberg.org')
    shakespeare_url = 'http://www.gutenberg.org/cache/epub/100/pg100.txt'
    # Get Shakespeare text
    response = requests.get(shakespeare_url)
    shakespeare_file = response.content
    # Decode binary into string
    s_text = shakespeare_file.decode('utf-8')
    # Drop first few descriptive paragraphs.
    s_text = s_text[7675:]
    # Remove newlines
    s_text = s_text.replace('\r\n', '')
    s_text = s_text.replace('\n', '')
    
    # Write to file
    with open(os.path.join(data_dir, data_file), 'w') as out_conn:
        out_conn.write(s_text)
else:
    # If file has been saved, load from that file
    with open(os.path.join(data_dir, data_file), 'r') as file_conn:
        s_text = file_conn.read().replace('\n', '')

print('Done Loading Data.')

Loading Shakespeare Data
Not found, downloading Shakespeare texts from www.gutenberg.org
Done Loading Data.


Clean and split the text data.

In [7]:
# Clean text
print('Cleaning Text')
s_text = re.sub(r'[{}]'.format(punctuation), ' ', s_text)
s_text = re.sub('\s+', ' ', s_text ).strip().lower()

# Split up by characters
char_list = list(s_text)

Cleaning Text


Build word vocabulary function and transform the text.

In [8]:
def build_vocab(characters):
    character_counts = collections.Counter(characters)
    # Create vocab --> index mapping
    chars = character_counts.keys()
    vocab_to_ix_dict = {key:(ix+1) for ix, key in enumerate(chars)}
    # Add unknown key --> 0 index
    vocab_to_ix_dict['unknown']=0
    # Create index --> vocab mapping
    ix_to_vocab_dict = {val:key for key,val in vocab_to_ix_dict.items()}
    return(ix_to_vocab_dict, vocab_to_ix_dict)

# Build Shakespeare vocabulary
print('Building Shakespeare Vocab by Characters')
ix2vocab, vocab2ix = build_vocab(char_list)
vocab_size = len(ix2vocab)
print('Vocabulary Length = {}'.format(vocab_size))
# Sanity Check
assert(len(ix2vocab) == len(vocab2ix))

Building Shakespeare Vocab by Characters
Vocabulary Length = 40


Convert text to word vectors

In [9]:
s_text_ix = []
for x in char_list:
    try:
        s_text_ix.append(vocab2ix[x])
    except:
        s_text_ix.append(0)
s_text_ix = np.array(s_text_ix)

Define LSTM RNN Model Class

In [10]:
class LSTM_Model():
    def __init__(self, rnn_size, num_layers, batch_size, learning_rate,
                 training_seq_len, vocab_size, infer_sample=False):
        self.rnn_size = rnn_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size
        self.infer_sample = infer_sample
        self.learning_rate = learning_rate
        
        if infer_sample:
            self.batch_size = 1
            self.training_seq_len = 1
        else:
            self.batch_size = batch_size
            self.training_seq_len = training_seq_len
        
        self.lstm_cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        self.lstm_cell = tf.contrib.rnn.MultiRNNCell([self.lstm_cell for _ in range(self.num_layers)])
        self.initial_state = self.lstm_cell.zero_state(self.batch_size, tf.float32)
        
        self.x_data = tf.placeholder(tf.int32, [self.batch_size, self.training_seq_len])
        self.y_output = tf.placeholder(tf.int32, [self.batch_size, self.training_seq_len])
        
        with tf.variable_scope('lstm_vars'):
            # Softmax Output Weights
            W = tf.get_variable('W', [self.rnn_size, self.vocab_size], tf.float32, tf.random_normal_initializer())
            b = tf.get_variable('b', [self.vocab_size], tf.float32, tf.constant_initializer(0.0))
        
            # Define Embedding
            embedding_mat = tf.get_variable('embedding_mat', [self.vocab_size, self.rnn_size],
                                            tf.float32, tf.random_normal_initializer())
                                            
            embedding_output = tf.nn.embedding_lookup(embedding_mat, self.x_data)
            rnn_inputs = tf.split(axis=1, num_or_size_splits=self.training_seq_len, value=embedding_output)
            rnn_inputs_trimmed = [tf.squeeze(x, [1]) for x in rnn_inputs]
        
        decoder = tf.contrib.legacy_seq2seq.rnn_decoder
        outputs, last_state = decoder(rnn_inputs_trimmed,
                                      self.initial_state,
                                      self.lstm_cell)
        
        # RNN outputs
        output = tf.reshape(tf.concat(axis=1, values=outputs), [-1, rnn_size])
        # Logits and output
        self.logit_output = tf.matmul(output, W) + b
        self.model_output = tf.nn.softmax(self.logit_output)
        
        loss_fun = tf.contrib.legacy_seq2seq.sequence_loss_by_example
        loss = loss_fun([self.logit_output],[tf.reshape(self.y_output, [-1])],
                [tf.ones([self.batch_size * self.training_seq_len])],
                self.vocab_size)
        self.cost = tf.reduce_sum(loss) / (self.batch_size * self.training_seq_len)
        self.final_state = last_state
        gradients, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tf.trainable_variables()), 4.5)
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        self.train_op = optimizer.apply_gradients(zip(gradients, tf.trainable_variables()))
        
    def sample(self, sess, words=ix2vocab, vocab=vocab2ix, num=20, prime_text='thou art'):
        state = sess.run(self.lstm_cell.zero_state(1, tf.float32))
        char_list = list(prime_text)
        for char in char_list[:-1]:
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed_dict = {self.x_data: x, self.initial_state:state}
            [state] = sess.run([self.final_state], feed_dict=feed_dict)

        out_sentence = prime_text
        char = char_list[-1]
        for n in range(num):
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed_dict = {self.x_data: x, self.initial_state:state}
            [model_output, state] = sess.run([self.model_output, self.final_state], feed_dict=feed_dict)
            sample = np.argmax(model_output[0])
            if sample == 0:
                break
            char = words[sample]
            out_sentence = out_sentence + char
        return(out_sentence)

Initialize the LSTM Model

In [11]:
lstm_model = LSTM_Model(rnn_size, num_layers, batch_size, learning_rate,
                        training_seq_len, vocab_size)

# Tell TensorFlow we are reusing the scope for the testing
with tf.variable_scope(tf.get_variable_scope(), reuse=True):
    test_lstm_model = LSTM_Model(rnn_size,num_layers, batch_size, learning_rate,
                                 training_seq_len, vocab_size, infer_sample=True)

Create model saver

In [12]:
saver = tf.train.Saver(tf.global_variables())

Create batches for each epoch

In [13]:
num_batches = int(len(s_text_ix)/(batch_size * training_seq_len)) + 1
# Split up text indices into subarrays, of equal size
batches = np.array_split(s_text_ix, num_batches)
# Reshape each split into [batch_size, training_seq_len]
batches = [np.resize(x, [batch_size, training_seq_len]) for x in batches]

Initialize all variables and train the model!

In [14]:
# Initialize all variables
init = tf.global_variables_initializer()
sess.run(init)

# Train model
train_loss = []
iteration_count = 1
for epoch in range(epochs):
    # Shuffle word indices
    random.shuffle(batches)
    # Create targets from shuffled batches
    targets = [np.roll(x, -1, axis=1) for x in batches]
    # Run a through one epoch
    print('Starting Epoch #{} of {}.'.format(epoch+1, epochs))
    # Reset initial LSTM state every epoch
    state = sess.run(lstm_model.initial_state)
    for ix, batch in enumerate(batches):
        training_dict = {lstm_model.x_data: batch, lstm_model.y_output: targets[ix]}
        # We need to update initial state for each RNN cell:
        for i, (c, h) in enumerate(lstm_model.initial_state):
                    training_dict[c] = state[i].c
                    training_dict[h] = state[i].h
        
        temp_loss, state, _ = sess.run([lstm_model.cost, lstm_model.final_state, lstm_model.train_op],
                                       feed_dict=training_dict)
        train_loss.append(temp_loss)
        
        # Print status every 10 gens
        if iteration_count % 10 == 0:
            summary_nums = (iteration_count, epoch+1, ix+1, num_batches+1, temp_loss)
            print('Iteration: {}, Epoch: {}, Batch: {} out of {}, Loss: {:.2f}'.format(*summary_nums))
        
        # Save the model and the vocab
        if iteration_count % save_every == 0:
            # Save model
            model_file_name = os.path.join(full_model_dir, 'model')
            saver.save(sess, model_file_name, global_step = iteration_count)
            print('Model Saved To: {}'.format(model_file_name))
            # Save vocabulary
            dictionary_file = os.path.join(full_model_dir, 'vocab.pkl')
            with open(dictionary_file, 'wb') as dict_file_conn:
                pickle.dump([vocab2ix, ix2vocab], dict_file_conn)
        
        if iteration_count % eval_every == 0:
            for sample in prime_texts:
                print(test_lstm_model.sample(sess, ix2vocab, vocab2ix, num=10, prime_text=sample))
                
        iteration_count += 1

Starting Epoch #1 of 10.
Iteration: 10, Epoch: 1, Batch: 10 out of 950, Loss: 3.06
Iteration: 20, Epoch: 1, Batch: 20 out of 950, Loss: 2.95
Iteration: 30, Epoch: 1, Batch: 30 out of 950, Loss: 2.88
Iteration: 40, Epoch: 1, Batch: 40 out of 950, Loss: 2.87
Iteration: 50, Epoch: 1, Batch: 50 out of 950, Loss: 2.85
thou art more    ee o o
to be or not to   ee  e o
wherefore art thou   ee  oo 
Iteration: 60, Epoch: 1, Batch: 60 out of 950, Loss: 2.84
Iteration: 70, Epoch: 1, Batch: 70 out of 950, Loss: 2.84
Iteration: 80, Epoch: 1, Batch: 80 out of 950, Loss: 2.83
Iteration: 90, Epoch: 1, Batch: 90 out of 950, Loss: 2.79
Iteration: 100, Epoch: 1, Batch: 100 out of 950, Loss: 2.79
thou art more  teo o oo
to be or not to e hee o o
wherefore art thou  eoo o oo
Iteration: 110, Epoch: 1, Batch: 110 out of 950, Loss: 2.77
Iteration: 120, Epoch: 1, Batch: 120 out of 950, Loss: 2.77
Iteration: 130, Epoch: 1, Batch: 130 out of 950, Loss: 2.73
Iteration: 140, Epoch: 1, Batch: 140 out of 950, Loss: 

Type is unsupported, or the types of the items don't match field type in CollectionDef.
'dict' object has no attribute 'name'


Model Saved To: temp/shakespeare_model/model
thou art more the the t
to be or not to the the t
wherefore art thou hat the t
Iteration: 510, Epoch: 1, Batch: 510 out of 950, Loss: 2.24
Iteration: 520, Epoch: 1, Batch: 520 out of 950, Loss: 2.16
Iteration: 530, Epoch: 1, Batch: 530 out of 950, Loss: 2.15
Iteration: 540, Epoch: 1, Batch: 540 out of 950, Loss: 2.19
Iteration: 550, Epoch: 1, Batch: 550 out of 950, Loss: 2.14
thou art more the the t
to be or not to the the t
wherefore art thou the the t
Iteration: 560, Epoch: 1, Batch: 560 out of 950, Loss: 2.12
Iteration: 570, Epoch: 1, Batch: 570 out of 950, Loss: 2.12
Iteration: 580, Epoch: 1, Batch: 580 out of 950, Loss: 2.17
Iteration: 590, Epoch: 1, Batch: 590 out of 950, Loss: 2.11
Iteration: 600, Epoch: 1, Batch: 600 out of 950, Loss: 2.25
thou art more the the t
to be or not to me the th
wherefore art thou the the t
Iteration: 610, Epoch: 1, Batch: 610 out of 950, Loss: 2.07
Iteration: 620, Epoch: 1, Batch: 620 out of 950, Loss: 2.0

Type is unsupported, or the types of the items don't match field type in CollectionDef.
'dict' object has no attribute 'name'


Model Saved To: temp/shakespeare_model/model
thou art more the man t
to be or not to man the m
wherefore art thou have the 
Iteration: 1010, Epoch: 2, Batch: 61 out of 950, Loss: 2.05
Iteration: 1020, Epoch: 2, Batch: 71 out of 950, Loss: 2.01
Iteration: 1030, Epoch: 2, Batch: 81 out of 950, Loss: 1.98
Iteration: 1040, Epoch: 2, Batch: 91 out of 950, Loss: 1.98
Iteration: 1050, Epoch: 2, Batch: 101 out of 950, Loss: 1.94
thou art more the the t
to be or not to the the s
wherefore art thou her the t
Iteration: 1060, Epoch: 2, Batch: 111 out of 950, Loss: 2.02
Iteration: 1070, Epoch: 2, Batch: 121 out of 950, Loss: 2.15
Iteration: 1080, Epoch: 2, Batch: 131 out of 950, Loss: 2.02
Iteration: 1090, Epoch: 2, Batch: 141 out of 950, Loss: 1.95
Iteration: 1100, Epoch: 2, Batch: 151 out of 950, Loss: 1.92
thou art more the the t
to be or not to the the t
wherefore art thou the the t
Iteration: 1110, Epoch: 2, Batch: 161 out of 950, Loss: 1.99
Iteration: 1120, Epoch: 2, Batch: 171 out of 950, L

...

Model Saved To: temp/shakespeare_model/model
thou art more the senat
to be or not to the senat
wherefore art thou shall see
Iteration: 8010, Epoch: 9, Batch: 418 out of 950, Loss: 1.52
Iteration: 8020, Epoch: 9, Batch: 428 out of 950, Loss: 1.62
Iteration: 8030, Epoch: 9, Batch: 438 out of 950, Loss: 1.59
Iteration: 8040, Epoch: 9, Batch: 448 out of 950, Loss: 1.54
Iteration: 8050, Epoch: 9, Batch: 458 out of 950, Loss: 1.59
thou art more the send 
to be or not to the stand
wherefore art thou art the s
Iteration: 8060, Epoch: 9, Batch: 468 out of 950, Loss: 1.56
Iteration: 8070, Epoch: 9, Batch: 478 out of 950, Loss: 1.70
Iteration: 8080, Epoch: 9, Batch: 488 out of 950, Loss: 1.58
Iteration: 8090, Epoch: 9, Batch: 498 out of 950, Loss: 1.61
Iteration: 8100, Epoch: 9, Batch: 508 out of 950, Loss: 1.70
thou art more the see t
to be or not to the stran
wherefore art thou art the s
Iteration: 8110, Epoch: 9, Batch: 518 out of 950, Loss: 1.66
Iteration: 8120, Epoch: 9, Batch: 528 out of 95

Type is unsupported, or the types of the items don't match field type in CollectionDef.
'dict' object has no attribute 'name'


Model Saved To: temp/shakespeare_model/model
thou art more the send 
to be or not to the stran
wherefore art thou art the s
Iteration: 8510, Epoch: 9, Batch: 918 out of 950, Loss: 1.62
Iteration: 8520, Epoch: 9, Batch: 928 out of 950, Loss: 1.62
Iteration: 8530, Epoch: 9, Batch: 938 out of 950, Loss: 1.58
Iteration: 8540, Epoch: 9, Batch: 948 out of 950, Loss: 1.58
Starting Epoch #10 of 10.
Iteration: 8550, Epoch: 10, Batch: 9 out of 950, Loss: 1.67
thou art more the world
to be or not to the compl
wherefore art thou shall be 
Iteration: 8560, Epoch: 10, Batch: 19 out of 950, Loss: 1.66
Iteration: 8570, Epoch: 10, Batch: 29 out of 950, Loss: 1.53
Iteration: 8580, Epoch: 10, Batch: 39 out of 950, Loss: 1.57
Iteration: 8590, Epoch: 10, Batch: 49 out of 950, Loss: 1.69
Iteration: 8600, Epoch: 10, Batch: 59 out of 950, Loss: 1.60
thou art more the see t
to be or not to the see t
wherefore art thou shall be 
Iteration: 8610, Epoch: 10, Batch: 69 out of 950, Loss: 1.64
Iteration: 8620, Epoch

Type is unsupported, or the types of the items don't match field type in CollectionDef.
'dict' object has no attribute 'name'


Model Saved To: temp/shakespeare_model/model
thou art more that thou
to be or not to the compl
wherefore art thou art thou 
Iteration: 9010, Epoch: 10, Batch: 469 out of 950, Loss: 1.67
Iteration: 9020, Epoch: 10, Batch: 479 out of 950, Loss: 1.57
Iteration: 9030, Epoch: 10, Batch: 489 out of 950, Loss: 1.71
Iteration: 9040, Epoch: 10, Batch: 499 out of 950, Loss: 1.65
Iteration: 9050, Epoch: 10, Batch: 509 out of 950, Loss: 1.63
thou art more the world
to be or not to the world
wherefore art thou shall be 
Iteration: 9060, Epoch: 10, Batch: 519 out of 950, Loss: 1.62
Iteration: 9070, Epoch: 10, Batch: 529 out of 950, Loss: 1.51
Iteration: 9080, Epoch: 10, Batch: 539 out of 950, Loss: 1.61
Iteration: 9090, Epoch: 10, Batch: 549 out of 950, Loss: 1.48
Iteration: 9100, Epoch: 10, Batch: 559 out of 950, Loss: 1.68
thou art more the stand
to be or not to the compl
wherefore art thou art the s
Iteration: 9110, Epoch: 10, Batch: 569 out of 950, Loss: 1.59
Iteration: 9120, Epoch: 10, Batch: 5

Plot loss over time

In [16]:
plt.plot(train_loss, 'k-')
plt.title('Sequence to Sequence Loss')
plt.xlabel('Generation')
plt.ylabel('Loss')
plt.show()